In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow import keras
import kerastuner as kt
import numpy as np

In [3]:
train_images = np.load(r"rds/projects/s/stevenir-radio-astronomy-neural-nets/data/train_images.npy")
test_images = np.load(r"rds/projects/s/stevenir-radio-astronomy-neural-nets/data/test_images.npy")
train_labels = np.load(r"rds/projects/s/stevenir-radio-astronomy-neural-nets/data/train_labels.npy")
test_labels = np.load(r"rds/projects/s/stevenir-radio-astronomy-neural-nets/data/test_labels.npy")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [4]:
input_shape = train_images.shape[1:]

def build_model(hp):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 10, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 128, step=32)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D(padding='same')(x)
        else:
            x = tf.keras.layers.AvgPool2D( padding='same')(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(hp.Int('hidden_size', 30, 100, step=10, default=50), activation='relu')(x)
    x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(1, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-3, 1e-2, sampling='log')),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [5]:
tuner = kt.Hyperband(build_model,objective='val_accuracy',max_epochs=30,hyperband_iterations=2,directory='my_dir',project_name='intro_to_kt')

In [ ]:
batch_size = 32
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50)

tuner.search(train_ds.batch(batch_size),validation_data=test_ds.batch(batch_size),epochs=30,callbacks=[callback])

In [23]:
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [ ]:
model.save(r"/rds/projects/s/stevenir-radio-astronomy-neural-nets/Sharif/modelBlueBEAR")